<a href="https://colab.research.google.com/github/Danila-Kovalenko/My_Machine_Learning/blob/main/gold_gewinnung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gewinnung von Gold aus Erz

Es ist notwendig, einen Prototyp eines maschinellen Lernmodells vorzubereiten, das die Gewinnungsrate von Gold aus Golderz vorhersagen soll. Verwenden Sie Daten mit Bergbau- und Reinigungsparametern.

Das Modell wird dazu beitragen, die Produktion zu optimieren, um kein Unternehmen mit unrentablen Eigenschaften zu gründen.


## Vorbereitung

In [ ]:
# Wir importieren die notwendigen Bibliotheken.
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.metrics.scorer import make_scorer
from sklearn.model_selection import GridSearchCV

from scipy import stats as st

In [ ]:
# Wir laden Tabellen mit Daten
df_train = pd.read_csv('C:/Users/Konstantin/Documents/datasets/Gold Recovery/gold_recovery_train_new.csv')
df_test = pd.read_csv('C:/Users/Konstantin/Documents/datasets/Gold Recovery/gold_recovery_test_new.csv')
df_full = pd.read_csv('C:/Users/Konstantin/Documents/datasets/Gold Recovery/gold_recovery_full_new.csv')

In [ ]:
# Aufgrund der Tatsache, dass die Daten in benachbarten Zeilen einen ähnlichen Wert haben,
# füllen wir die Nulldaten mit Werten aus benachbarten Zeilen aus.
for file in datas:
    file.ffill(axis = 0, inplace = True)
    display(file.info())

In [ ]:
# Wir berechnen die Goldanreicherungseffizienz auf dem Trainingsset.
C = df_train['rougher.output.concentrate_au']
F = df_train['rougher.input.feed_au']
T = df_train['rougher.output.tail_au']

rougher_output_recovery = pd.Series(C*(F-T)*100/F/(C-T))

display(df_train['rougher.output.recovery'].head())
display(rougher_output_recovery.head())

In [ ]:
# Wir berechnen die durchschnittliche Effizienz der Goldanreicherung.
print('Durchschnittliche Effizienz der Goldanreicherung angegeben', df_train['rougher.output.recovery'].mean())
print('Durchschnittliche Effizienz der Goldanreicherung berechnet', rougher_output_recovery.mean())

In [ ]:
# Wir berechnen die durchschnittliche absolute Abweichung zwischen der angegebenen und
# der berechneten durchschnittlichen Goldeffizienz.
print('Durchschnittliche absolute Abweichung', mean_absolute_error(df_train['rougher.output.recovery'], rougher_output_recovery))

**Schlussfolgerungen**

Wir haben eine Datenvorverarbeitung durchgeführt, indem wir fehlende Werte mit den Daten benachbarter Zeilen ergänzt haben. Anschließend haben wir die Effizienz der Goldanreicherung berechnet. Die Durchschnittswerte zeigten keine Abweichung. Die mittlere absolute Abweichung lag nahe Null, was höchstwahrscheinlich darauf zurückzuführen ist, dass die fehlenden Werte mit möglicherweise ungenauen Daten ersetzt wurden. Dies deutet darauf hin, dass die Berechnung der mittleren absoluten Abweichung in der angegebenen Tabelle korrekt durchgeführt wurde.

## Explorative Datenanalyse

In [ ]:
# Wir werden sehen, wie sich die Konzentration von Metallen (Au, Ag, Pb) in verschiedenen Stadien der Reinigung ändert.
print('Au')
print(df_train['rougher.input.feed_au'].mean())
print(df_train['rougher.output.concentrate_au'].mean())
print(df_train['primary_cleaner.output.concentrate_au'].mean())
print(df_train['final.output.concentrate_au'].mean())
print()

print('Ag')
print(df_train['rougher.input.feed_ag'].mean())
print(df_train['rougher.output.concentrate_ag'].mean())
print(df_train['primary_cleaner.output.concentrate_ag'].mean())
print(df_train['final.output.concentrate_ag'].mean())
print()

print('Pb')
print(df_train['rougher.input.feed_pb'].mean())
print(df_train['rougher.output.concentrate_pb'].mean())
print(df_train['primary_cleaner.output.concentrate_pb'].mean())
print(df_train['final.output.concentrate_pb'].mean())
print()

**Schlussfolgerungen**

Nachdem wir die durchschnittliche Konzentration von Metallen in verschiedenen Reinigungsstufen berechnet haben, sehen wir, dass die Konzentration von Gold von Stufe zu Stufe erheblich zunimmt, die Konzentration von Silber zuerst leicht zunimmt, dann abnimmt, die Konzentration von Blei zunimmt, aber nicht signifikant.

In [ ]:
# Wir untersuchen die Gesamtkonzentration aller Substanzen in verschiedenen Stadien.
print((df_train['rougher.input.feed_au']+df_train['rougher.input.feed_ag']+
       df_train['rougher.input.feed_pb']+df_train['rougher.input.feed_sol']).mean())

print((df_train['rougher.output.concentrate_au']+df_train['rougher.output.concentrate_ag']+
       df_train['rougher.output.concentrate_pb']+df_train['rougher.output.concentrate_sol']).mean())

print((df_train['primary_cleaner.output.concentrate_au']+df_train['primary_cleaner.output.concentrate_ag']+
       df_train['primary_cleaner.output.concentrate_pb']+df_train['primary_cleaner.output.concentrate_sol']).mean())

print((df_train['final.output.concentrate_au']+df_train['final.output.concentrate_ag']+
       df_train['final.output.concentrate_pb']+df_train['final.output.concentrate_sol']).mean())

**Schlussfolgerungen**

Nach der Analyse der Gesamtkonzentration aller Substanzen in verschiedenen Stufen sehen wir, dass nach der ersten Reinigungsstufe die Gesamtkonzentration abnimmt, aber nach der letzten Reinigungsstufe wieder ansteigt.

In [ ]:
# Wir analysieren die Diagramme der Größenverteilung der Rohstoffkörnchen auf den Trainings- und Testmustern.
df_train['rougher.input.feed_size'].hist(bins = 200, range=(20, 100))
df_test['rougher.input.feed_size'].hist(bins = 200, range=(20, 100))

In [ ]:
df_train['primary_cleaner.input.feed_size'].hist(bins = 100, range=(5, 9))
df_test['primary_cleaner.input.feed_size'].hist(bins = 100, range=(5, 9))

**Schlussfolgerungen**

Wir haben die Verteilungsdiagramme der Rohstoffgranulatgrößen an den Trainings- und Testmustern analysiert. Wir können sehen, dass der Datensatz in beiden Fällen rechtsschief ist. Die Diagramme sind ähnlich, daher unterscheiden sich die Verteilungen nicht sehr voneinander, die Schätzung des Modells wird korrekt sein.

## Modelltraining und Testen

In [ ]:
# Wir erstellen eine smape-Berechnungsfunktion.
def smape(target, predict):
    return (100/len(target)) * np.sum(2 * np.abs(target - predict) / (np.abs(target) + np.abs(predict)))

# Wir werden unsere eigene Metrik erstellen.
smape_scorer = make_scorer(smape, greater_is_better=True)

In [ ]:
# Wir vervollständigen die Testtabelle mit den fehlenden Zielparametern rougher.output.recovery und final.output.recovery.
df_test = df_test.merge(df_full[['date', 'rougher.output.recovery','final.output.recovery']], on='date')
df_test.info()

# Wir werden die Spalten, die sich im Test befinden, in der Trainingsstichprobe belassen.
df_train = df_train[df_test.columns]
df_train.info()

In [ ]:
# Wir werden Samples für den Zielparameter rougher.output.recovery generieren.
features_1 = df_train.loc[:, 'rougher.input.feed_ag': 'rougher.state.floatbank10_f_level']
target_1 = df_train['rougher.output.recovery']

In [ ]:
# Wir finden die besten Hyperparameter für das Modell DecisionTreeRegressor.
model = DecisionTreeRegressor()
parameters = {'max_depth':[9,10,11,12,13,14,15,25]}

grid = GridSearchCV(model, parameters, cv = 5, scoring = smape_scorer)
grid.fit(features_1, target_1)

grid.best_params_

In [ ]:
# Wir finden die besten Hyperparameter für das Modell RandomForestRegressor.
model = RandomForestRegressor()

parametrs = { 'n_estimators': range (30, 51, 10),
              'max_depth': range (9,13, 2) }

grid = GridSearchCV(model, parametrs, cv=5, scoring = smape_scorer)
grid.fit(features_1, target_1)

grid.best_params_

In [ ]:
# Wir finden die besten Hyperparameter für das Modell LinearRegression.
model = LinearRegression()
parametrs = {'fit_intercept' : [True, False], 'copy_X': [True, False], 'n_jobs' :[None, -1]}

grid = GridSearchCV(estimator = model, param_grid = parametrs, cv = 5, scoring = smape_scorer)
grid.fit(features_1, target_1)

grid.best_params_

In [ ]:
# Wir werden Stichproben für den Zielparameter final.output.recovery generieren.
features_2 = df_train.drop(['date', 'rougher.output.recovery', 'final.output.recovery'], axis=1)
target_2 = df_train['final.output.recovery']

In [ ]:
# Wir finden die besten Hyperparameter für das Modell DecisionTreeRegressor.
model = DecisionTreeRegressor()
parameters = {'max_depth':[160,161,162,163,164]}

grid = GridSearchCV(model, parameters, cv = 5, scoring = smape_scorer)
grid.fit(features_2, target_2)

grid.best_params_

In [ ]:
# Wir finden die besten Hyperparameter für das Modell RandomForestRegressor.
model = RandomForestRegressor()

parametrs = { 'n_estimators': range (10, 31, 10),
              'max_depth': range (5,11, 2) }

grid = GridSearchCV(model, parametrs, cv=5, scoring = smape_scorer)
grid.fit(features_2, target_2)

grid.best_params_

In [ ]:
# Wir finden die besten Hyperparameter für das Modell LinearRegression.
model = LinearRegression()
parametrs = {'fit_intercept' : [True, False], 'copy_X': [True, False], 'n_jobs' :[None, -1]}

grid = GridSearchCV(estimator = model, param_grid = parametrs, cv = 5, scoring = smape_scorer)
grid.fit(features_2, target_2)

grid.best_params_

In [ ]:
# Wir werden die smape-Modelle mit den besten Hyperparametern für das Zielmerkmal rougher.output.recovery berechnen.
features_train, features_valid, target_train, target_valid = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)

model_1_1 = DecisionTreeRegressor(random_state=12345, max_depth=13)
model_1_1.fit(features_train, target_train)
predictions_valid = model_1_1.predict(features_valid)

result_1_1 = smape(target_valid, predictions_valid)
print('DecisionTreeRegressor', result_1_1)


model_1_2 = RandomForestRegressor(random_state=12345, n_estimators=40, max_depth=11)
model_1_2.fit(features_train, target_train)
predictions_valid = model_1_2.predict(features_valid)

result_1_2 = smape(target_valid, predictions_valid)
print('RandomForestRegressor', result_1_2)


model_1_3 = LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None)
model_1_3.fit(features_train, target_train)
predictions_valid = model_1_3.predict(features_valid)

result_1_3 = smape(target_valid, predictions_valid)
print('LinearRegression',result_1_3)

In [ ]:
# Wir werden die smape-Modelle mit den besten Hyperparametern für das Zielmerkmal final.output.recovery berechnen.
features_train, features_valid, target_train, target_valid = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)

model_2_1 = DecisionTreeRegressor(random_state=12345, max_depth=161)
model_2_1.fit(features_train, target_train)
predictions_valid = model_2_1.predict(features_valid)

result_2_1 = smape(target_valid, predictions_valid)
print('DecisionTreeRegressor', result_2_1)


model_2_2 = RandomForestRegressor(random_state=12345, n_estimators=10, max_depth=9)
model_2_2.fit(features_train, target_train)
predictions_valid = model_2_2.predict(features_valid)

result_2_2 = smape(target_valid, predictions_valid)
print('RandomForestRegressor', result_2_2)


model_2_3 = LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None)
model_2_3.fit(features_train, target_train)
predictions_valid = model_2_3.predict(features_valid)

result_2_3 = smape(target_valid, predictions_valid)
print('LinearRegression', result_2_3)

In [ ]:
# Wir werden den endgültigen smape für die Trainingsprobe berechnen.
final_smape = 0.25 * result_1_1 + 0.75 * result_2_2
print('Das endgültige smape für die Trainingsprobe:', final_smape)

In [ ]:
# Wir werden die besten Modelle in einer Testprobe testen.

# Wir werden Stichproben für den Zielparameter von rougher.output.recovery.
features_1 = df_test.loc[:, 'rougher.input.feed_ag': 'rougher.state.floatbank10_f_level']
target_1 = df_test['rougher.output.recovery']

features_train, features_valid, target_train, target_valid = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)

model = DecisionTreeRegressor(random_state=12345, max_depth=130)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)

result_1 = smape(target_valid, predictions_valid)
print('DecisionTreeRegressor', result_1)

# Wir werden Stichproben für den Zielparameter von final.output.recovery.
features_2 = df_test.drop(['date', 'rougher.output.recovery', 'final.output.recovery'], axis=1)
target_2 = df_test['final.output.recovery']

features_train, features_valid, target_train, target_valid = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)

model = RandomForestRegressor(random_state=12345, n_estimators=10, max_depth=7)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)

result_2 = smape(target_valid, predictions_valid)
print('RandomForestRegressor', result_2)

In [ ]:
# Wir werden das endgültige smape für die Testprobe berechnen.
final_smape = 0.25 * result_1 + 0.75 * result_2
print('Das endgültige Smape für die Testprobe:', final_smape)

In [ ]:
# Wir werden Stichproben für den Zielparameter von rougher.output.recovery.
predictions_1 = model_1_1.predict(features_1)
result_1 = smape(target_1, predictions_1)
print('DecisionTreeRegressor', result_1)

predictions_2 = model_2_2.predict(features_2)
result_2 = smape(target_2, predictions_2)
print('DecisionTreeRegressor', result_2)

In [ ]:
# Wir werden das endgültige smape für die Testprobe berechnen.
final_smape = 0.25 * result_1 + 0.75 * result_2
print('Das endgültige Smape für die Testprobe:', final_smape)

**Schlussfolgerungen**

Unter Verwendung des GridSearchCV-Pakets haben wir die optimalen Hyperparameter für drei Modelltypen ausgewählt und Modelle mit der kleinsten Smape-Metrik ausgewählt.

Die ausgewählten Modelle wurden an einer Testprobe getestet, und der endgültige smape-Wert wurde für die Zwischen- und Endparameter der Golderzaufbereitung berechnet.